# Basic Agent Conversation with A2A

This notebook demonstrates how to use the Python A2A package to create conversations between agents. We'll explore:

1. Creating A2A messages
2. Building conversations
3. Exchanging messages between agents
4. Working with different message types

Let's get started!

## Setup

First, let's make sure we have the Python A2A package installed:

In [ ]:
!pip install python-a2a

Now, let's import the necessary components from the package:

In [ ]:
from python_a2a import (
    Message, TextContent, FunctionCallContent, FunctionResponseContent, FunctionParameter,
    MessageRole, Conversation,
    A2AClient, A2AServer,
    pretty_print_message, pretty_print_conversation
)

## Creating Messages

Let's start by creating a simple text message:

In [ ]:
# Create a text message from the user
text_message = Message(
    content=TextContent(text="What's the weather like in New York?"),
    role=MessageRole.USER
)

# Display the message
pretty_print_message(text_message)

We can also create function call messages, which are used to request specific actions:

In [ ]:
# Create a function call message
function_call = Message(
    content=FunctionCallContent(
        name="get_weather",
        parameters=[
            FunctionParameter(name="location", value="New York"),
            FunctionParameter(name="unit", value="celsius")
        ]
    ),
    role=MessageRole.AGENT
)

# Display the function call
pretty_print_message(function_call)

And function response messages, which return the results of a function call:

In [ ]:
# Create a function response message
function_response = Message(
    content=FunctionResponseContent(
        name="get_weather",
        response={
            "temperature": 22,
            "conditions": "Partly Cloudy",
            "humidity": 65,
            "wind_speed": 8
        }
    ),
    role=MessageRole.AGENT,
    parent_message_id=function_call.message_id
)

# Display the function response
pretty_print_message(function_response)

## Building Conversations

Now let's build a conversation that includes multiple messages:

In [ ]:
# Create a new conversation
conversation = Conversation()

# Add messages to the conversation
conversation.add_message(text_message)
conversation.add_message(function_call)
conversation.add_message(function_response)

# Add an agent's text response
weather_response = Message(
    content=TextContent(
        text="The weather in New York is currently 22°C (71.6°F) and partly cloudy. "
             "The humidity is 65% with wind speeds of 8 km/h."
    ),
    role=MessageRole.AGENT,
    parent_message_id=function_response.message_id,
    conversation_id=conversation.conversation_id
)
conversation.add_message(weather_response)

# Display the conversation
pretty_print_conversation(conversation)

We can also use helper methods to create messages directly in the conversation:

In [ ]:
# Create a new conversation
conversation2 = Conversation()

# Add a user message
msg1 = conversation2.create_text_message(
    text="What's the weather like in New York?",
    role=MessageRole.USER
)

# Add a function call
msg2 = conversation2.create_function_call(
    name="get_weather",
    parameters=[
        {"name": "location", "value": "New York"},
        {"name": "unit", "value": "celsius"}
    ],
    role=MessageRole.AGENT,
    parent_message_id=msg1.message_id
)

# Add a function response
msg3 = conversation2.create_function_response(
    name="get_weather",
    response={
        "temperature": 22,
        "conditions": "Partly Cloudy",
        "humidity": 65,
        "wind_speed": 8
    },
    role=MessageRole.AGENT,
    parent_message_id=msg2.message_id
)

# Add an agent's text response
msg4 = conversation2.create_text_message(
    text="The weather in New York is currently 22°C (71.6°F) and partly cloudy. "
         "The humidity is 65% with wind speeds of 8 km/h.",
    role=MessageRole.AGENT,
    parent_message_id=msg3.message_id
)

# Display the conversation
pretty_print_conversation(conversation2)

## Serializing and Deserializing Messages

A2A messages and conversations can be serialized to JSON for transport and storage:

In [ ]:
# Serialize a message to JSON
json_message = text_message.to_json()
print(json_message)

In [ ]:
# Deserialize a message from JSON
deserialized_message = Message.from_json(json_message)
pretty_print_message(deserialized_message)

In [ ]:
# Serialize a conversation to JSON
json_conversation = conversation.to_json()
print(json_conversation[:500] + "...")

In [ ]:
# Deserialize a conversation from JSON
deserialized_conversation = Conversation.from_json(json_conversation)
pretty_print_conversation(deserialized_conversation)

## Creating a Simple Agent Server

Let's define a simple echo agent that responds to messages:

In [ ]:
class EchoAgent(A2AServer):
    """A simple agent that echoes back messages."""
    
    def handle_message(self, message):
        """Process incoming A2A messages."""
        if message.content.type == "text":
            return Message(
                content=TextContent(text=f"Echo: {message.content.text}"),
                role=MessageRole.AGENT,
                parent_message_id=message.message_id,
                conversation_id=message.conversation_id
            )
        else:
            return Message(
                content=TextContent(text=f"Received a {message.content.type} message type"),
                role=MessageRole.AGENT,
                parent_message_id=message.message_id,
                conversation_id=message.conversation_id
            )

# Create an instance of the echo agent
echo_agent = EchoAgent()

Now let's send a message to our echo agent:

In [ ]:
# Create a test message
test_message = Message(
    content=TextContent(text="Hello, agent!"),
    role=MessageRole.USER
)

# Send the message to the echo agent
response = echo_agent.handle_message(test_message)

# Display the response
pretty_print_message(response)

We can also send a conversation to the agent:

In [ ]:
# Create a new conversation
test_conversation = Conversation()
test_conversation.create_text_message(
    text="Hello, agent!",
    role=MessageRole.USER
)

# Send the conversation to the echo agent
updated_conversation = echo_agent.handle_conversation(test_conversation)

# Display the updated conversation
pretty_print_conversation(updated_conversation)

## Running a Server and Connecting with a Client

In a real application, you would run the agent as a server using `run_server()`, and then connect to it using `A2AClient`. Here's how you would do that if the server was running:

```python
# Run the server (in a separate process)
run_server(echo_agent, host="0.0.0.0", port=5000)

# Connect with a client
client = A2AClient("http://localhost:5000/a2a")

# Send a message
response = client.send_message(test_message)
```

For this notebook, we'll just simulate the client-server interaction:

In [ ]:
# Simulate a client-server interaction
def simulate_client_server(agent, message):
    """Simulate a client sending a message to a server."""
    print("Client sending message:")
    pretty_print_message(message)
    
    print("\nServer processing message...\n")
    response = agent.handle_message(message)
    
    print("Client received response:")
    pretty_print_message(response)
    
    return response

In [ ]:
# Simulate a client interaction with our echo agent
client_message = Message(
    content=TextContent(text="What is the meaning of life?"),
    role=MessageRole.USER
)

server_response = simulate_client_server(echo_agent, client_message)

## Conclusion

In this notebook, we've learned how to:

1. Create different types of A2A messages (text, function calls, function responses)
2. Build and manage conversations
3. Serialize and deserialize messages and conversations
4. Create a simple agent that can handle messages
5. Simulate client-server interactions

The A2A protocol provides a standardized way for agents to communicate, making it easy to build modular AI systems where specialized agents can collaborate to solve complex problems.